# validate results

## TODO
what happened to tracing when I fixed probability calcs - it seems like all trip ids are attached when I add one by
hand below - why?

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [36]:
import os
import argparse
from datetime import datetime

import numpy as np
import pandas as pd
from numpy.random import default_rng
from statsmodels.stats.gof import powerdiscrepancy
from statsmodels.stats.proportion import multinomial_proportions_confint
import numba
import matplotlib.pyplot as plt

from activitysim.cli import run
from activitysim.core import inject

from fru_utils import run_trip_mode_choice, comp_mode_shares, mode_choice_for_trip

In [37]:
pd.set_option("max_columns", 500)

In [35]:
np.argmax(np.array([[1,2,3],[4,6,5],[7,8,9]]), axis=1)

array([2, 1, 2])

In [34]:
df_ = pd.DataFrame([[1,2,3],[4,6,5],[7,8,9]], columns=["a", "b", "c"])
display(df_)
ch_ = df_.idxmax(1)
display(ch_)

,a,b,c
0,1,2,3
1,4,6,5
2,7,8,9


0    c
1    b
2    c
dtype: object

In [5]:
root_dir = "/mnt/c/Users/jan.zill/code/activitysim"
example_dir = os.path.join(root_dir, "test_example_mtc")

In [6]:
os.chdir(example_dir)

In [7]:
parser = argparse.ArgumentParser()
run.add_run_args(parser)
args = parser.parse_args(['-c', 'configs', '-o', 'output', '-d', 'data'])
#run.run(args)  # 2mins full example run
if not inject.is_injectable('preload_injectables'):
    from activitysim import abm  # register abm steps and other abm-specific injectables
run.handle_standard_args(args)  # possibly update injectables

In [69]:
from activitysim.core import inject
from activitysim.core import pipeline
from activitysim.core import config
from activitysim.core import expressions
from activitysim.core import simulate
from activitysim.abm.models.util import estimation

def run_mandatory_tour_frequency(choose_individual_max_utility=False):
    resume_after = "cdap_simulate"
    model_name = "mandatory_tour_frequency"
    chunk_size = 0  # test_mtc means no chunking

    pipeline.open_pipeline(resume_after)
    # preload any bulky injectables (e.g. skims) not in pipeline
    inject.get_injectable('preload_injectables', None)
    pipeline._PIPELINE.rng().begin_step(model_name)
    #step_name = model_name
    args = {}
    #checkpoint = pipeline.intermediate_checkpoint(model_name)
    inject.set_step_args(args)

    #network_los = inject.get_injectable('network_los')
    persons_merged = inject.get_table('persons_merged')

    trace_label = 'mandatory_tour_frequency'
    model_settings_file_name = 'mandatory_tour_frequency.yaml'
    model_settings = config.read_model_settings(model_settings_file_name)
    choosers = persons_merged.to_frame()
    # filter based on results of CDAP
    choosers = choosers[choosers.cdap_activity == 'M']
    print("Running mandatory_tour_frequency with %d persons", len(choosers))
    preprocessor_settings = model_settings.get('preprocessor', None)
    if preprocessor_settings:
        locals_dict = {}
        expressions.assign_columns(
            df=choosers,
            model_settings=preprocessor_settings,
            locals_dict=locals_dict,
            trace_label=trace_label)

    estimator = estimation.manager.begin_estimation('mandatory_tour_frequency')
    model_spec = simulate.read_model_spec(file_name=model_settings['SPEC'])
    coefficients_df = simulate.read_model_coefficients(model_settings)
    model_spec = simulate.eval_coefficients(model_spec, coefficients_df, estimator)
    nest_spec = config.get_logit_model_settings(model_settings)
    constants = config.get_model_constants(model_settings)
    choices = simulate.simple_simulate(
        choosers=choosers,
        spec=model_spec,
        nest_spec=nest_spec,
        locals_d=constants,
        chunk_size=chunk_size,
        trace_label=trace_label,
        trace_choice_name='mandatory_tour_frequency',
        estimator=estimator,
        choose_individual_max_utility=choose_individual_max_utility)

    # convert indexes to alternative names
    choices = pd.Series(model_spec.columns[choices.values], index=choices.index)

    finalise = True
    if finalise:
        inject.set_step_args(None)
        #
        pipeline._PIPELINE.rng().end_step(model_name)
        pipeline.add_checkpoint(model_name)
        if not pipeline.intermediate_checkpoint():
            pipeline.add_checkpoint(pipeline.FINAL_CHECKPOINT_NAME)

        pipeline.close_pipeline()

    return choices

In [70]:
#pipeline.close_pipeline()

In [71]:
ch_prev = run_mandatory_tour_frequency(choose_individual_max_utility=False)
ch_new = run_mandatory_tour_frequency(choose_individual_max_utility=True)

estimation bundle mandatory_tour_frequency not in settings file estimation.yaml


Running mandatory_tour_frequency with %d persons 89


estimation bundle mandatory_tour_frequency not in settings file estimation.yaml


Running mandatory_tour_frequency with %d persons 89


In [72]:
ch_prev.value_counts().to_frame("count_prev").merge(
    ch_new.value_counts().to_frame("count_new"), how="outer", left_index=True, right_index=True
).fillna(0)

,count_prev,count_new
school1,22,22.0
work1,61,65.0
work2,2,2.0
work_and_school,4,0.0


In [73]:
inject.get_injectable('settings')

{'input_table_list': [{'tablename': 'households',
   'filename': 'households.csv',
   'index_col': 'household_id',
   'rename_columns': {'HHID': 'household_id',
    'PERSONS': 'hhsize',
    'workers': 'num_workers',
    'VEHICL': 'auto_ownership',
    'TAZ': 'home_zone_id'},
   'keep_columns': ['home_zone_id',
    'income',
    'hhsize',
    'HHT',
    'auto_ownership',
    'num_workers']},
  {'tablename': 'persons',
   'filename': 'persons.csv',
   'index_col': 'person_id',
   'rename_columns': {'PERID': 'person_id'},
   'keep_columns': ['household_id',
    'age',
    'PNUM',
    'sex',
    'pemploy',
    'pstudent',
    'ptype']},
  {'tablename': 'land_use',
   'filename': 'land_use.csv',
   'index_col': 'zone_id',
   'rename_columns': {'TAZ': 'zone_id', 'COUNTY': 'county_id'},
   'keep_columns': ['DISTRICT',
    'SD',
    'county_id',
    'TOTHH',
    'TOTPOP',
    'TOTACRE',
    'RESACRE',
    'CIACRE',
    'TOTEMP',
    'AGE0519',
    'RETEMPN',
    'FPSEMPN',
    'HEREMPN',
    '

In [74]:
resume_after = "cdap_simulate"
model_name = "mandatory_tour_frequency"
chunk_size = 0  # test_mtc means no chunking

pipeline.open_pipeline(resume_after)
# preload any bulky injectables (e.g. skims) not in pipeline
inject.get_injectable('preload_injectables', None)
pipeline._PIPELINE.rng().begin_step(model_name)
#step_name = model_name
args = {}
#checkpoint = pipeline.intermediate_checkpoint(model_name)
inject.set_step_args(args)

#network_los = inject.get_injectable('network_los')
persons_merged = inject.get_table('persons_merged')

trace_label = 'mandatory_tour_frequency'
model_settings_file_name = 'mandatory_tour_frequency.yaml'
model_settings = config.read_model_settings(model_settings_file_name)
choosers = persons_merged.to_frame()
# filter based on results of CDAP
choosers = choosers[choosers.cdap_activity == 'M']
print("Running mandatory_tour_frequency with %d persons", len(choosers))
preprocessor_settings = model_settings.get('preprocessor', None)
if preprocessor_settings:
    locals_dict = {}
    expressions.assign_columns(
        df=choosers,
        model_settings=preprocessor_settings,
        locals_dict=locals_dict,
        trace_label=trace_label)

estimator = estimation.manager.begin_estimation('mandatory_tour_frequency')
model_spec = simulate.read_model_spec(file_name=model_settings['SPEC'])
coefficients_df = simulate.read_model_coefficients(model_settings)
model_spec = simulate.eval_coefficients(model_spec, coefficients_df, estimator)
nest_spec = config.get_logit_model_settings(model_settings)
constants = config.get_model_constants(model_settings)
choices = simulate.simple_simulate(
    choosers=choosers,
    spec=model_spec,
    nest_spec=nest_spec,
    locals_d=constants,
    chunk_size=chunk_size,
    trace_label=trace_label,
    trace_choice_name='mandatory_tour_frequency',
    estimator=estimator,
    choose_individual_max_utility=True)

# convert indexes to alternative names
choices = pd.Series(model_spec.columns[choices.values], index=choices.index)

finalise = True
if finalise:
    inject.set_step_args(None)
    #
    pipeline._PIPELINE.rng().end_step(model_name)
    pipeline.add_checkpoint(model_name)
    if not pipeline.intermediate_checkpoint():
        pipeline.add_checkpoint(pipeline.FINAL_CHECKPOINT_NAME)

    pipeline.close_pipeline()


estimation bundle mandatory_tour_frequency not in settings file estimation.yaml


Running mandatory_tour_frequency with %d persons 89


In [58]:
df_ = pd.DataFrame([[1,2,3],[4,6,5],[7,8,9]], columns=["a", "b", "c"])
display(df_)

,a,b,c
0,1,2,3
1,4,6,5
2,7,8,9


In [64]:
{v: k for k,v in enumerate(df_.columns)}

{'a': 0, 'b': 1, 'c': 2}